# Distributed Computation

## Quick Recap - Multiprocessing vs. Multithreading

Multiprocessing can utilize multiple CPU cores, thus achieving a more authentic sense of parallel computation. However, multiprocessing suffers when they need to share a common memory space.

On the other hand, multithreading can share a common memory space and achieve a more loose sense of parallel computation. As a result, multithreading is more like hyper-jumping through multiple queues (within the same process) while waiting for each particular thread's turn:
1. When it is one specific thread's turn, it will acquire the Global Interpreter Lock (GIL) to control the memory space _and_ the CPU core until it finishes its designated computation or hits another "busy-waiting" block (such as I/O actions).
2. Then, the loop releases the GIL and lets the jump proceed (context-switch) to the following thread.
3. Rinse and repeat (until done).

You can revisit [part 13](./13-data-processing.ipynb) for more details on this subject.

### Asynchronous I/O Loops, and Multithreading's Little Brother - Coroutines

Both multiprocessing and multithreading require dedicated hardware or operating system support. As software technologies mature, engineers started to explore capabilities within the application layer itself. As a result, the same conceptual model of multithreading gets a new interpretation within the programming stack (such as Python, or more appropriately, the CPython runtime), giving more direct control to the program within the runtime instead of relying on the OS mechanism to switch context between threads. The term [_Coroutine_](https://en.wikipedia.org/wiki/Coroutine), first coined in 1958, is a materialization of the concept of lightweight threads.

Both multithreading and coroutine techniques are suitable for I/O focused tasks, such as reading files from a disk or making HTTP requests. However, coroutines tend to require less computing resource overhead than threads by giving up some performance benefit from tapping into the more OS-native mechanisms.

In [4]:
import random
import requests


# 5 requests, each with delays ranging from 1-3 seconds
reqs = [
    f'https://httpbin.org/delay/{random.randint(1, 3)}'
    for _ in range(5)
]

def get_sync():
    all_data = []
    for req in reqs:
        res = requests.get(req)
        all_data.append(res.json())

    return all_data

In [5]:
%time res = get_sync()
print(res[-1], len(res))

CPU times: user 84.2 ms, sys: 6.88 ms, total: 91.1 ms
Wall time: 11.7 s
{'args': {}, 'data': '', 'files': {}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.25.1', 'X-Amzn-Trace-Id': 'Root=1-60d2651e-364946872247cadd59364ddd'}, 'origin': '107.179.188.69', 'url': 'https://httpbin.org/delay/1'} 5


In Python, [the `asyncio` module](https://docs.python.org/3/library/asyncio.html) provides a standard set of APIs for its users to utilize coroutines.

In [6]:
import asyncio

import aiohttp


async def get(session, url):
    res = await session.request('GET', url=url)
    data = await res.json()
    return data


async def main():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for req in reqs:
            tasks.append(get(session, url=req))

        all_data = await asyncio.gather(*tasks, return_exceptions=True)
        return all_data

There are two caveats about utilizing `asyncio` in the Jupyter Notebook environment:
1. We cannot use the %time magic command for async functions.
2. We cannot initiate an explicit event loop (it's already in one).

In [7]:
import time

# loop = asyncio.get_event_loop()
# loop.run_until_complete(main())
start = time.time()
res = await main()
print(round(time.time() - start, 1), 'seconds')
print(res[-1], len(res))

3.3 seconds
{'args': {}, 'data': '', 'files': {}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'Python/3.8 aiohttp/3.7.4.post0', 'X-Amzn-Trace-Id': 'Root=1-60d26534-7369e48c66ce404f66172d19'}, 'origin': '107.179.188.69', 'url': 'https://httpbin.org/delay/1'} 5


Let's use the responses to verify what the delays were in the API calls by using functional techniques `map()` and `reduce()` to extract delays (in seconds) and sum them up.

In [8]:
delays = [*map(lambda url: int(url.split('/')[-1]), reqs)]
delays

[3, 1, 3, 2, 1]

In [9]:
from functools import reduce

total_delay = reduce(lambda left, right: left + right, [*delays], 0)
total_delay

10

The theoretical total delay matches our observation from the synchronous process, while the _maximum_ from individual delays matches our observation from the asynchronous process.

## MapReduce - Distributing Computing Resources

Recall one of the most critical disadvantages that multiprocessing - the lack of a shared memory space. This particular constraint makes attempt to perform parrallel computation across multiple cores on a single machine often less desirable. One workaround of such issue is to leverage a datastore (such as files or databases backed by harddrives) as an inter-process data pool so that multiple processes can simultaneously read and write to it, such as a local SQLite database, as demonstrated in [a previous part](./11-work-with-sql.ipynb).

If we extend this problem to a larger scale, where not only the dataset we want to work with exceed the available memory space, but also impossible to efficiently store (or at all) on the disk of a single machine, then we need to revisit viable solutions.

The MapReduce model is a _divide and conquer_ strategy applying and extending the functional programming concepts of `map()` and `reduce()`, where a large dataset is dissected and distributed through a mapping procedure onto a multitude of "commodity" server nodes to parrallize the computation of the smaller portion, then reducing the resulting subsets back to less and less nodes until the final outcome is completed.

This model allows batch data processing to have near infinite capacity, and a relatively cost-effective way to speed up the process.

It is worth knowing that the recent development of cheaper and faster harddrives, especially the more wide-spread adoption of SSDs (solid-state drive), plays a vital role in enabling distributed computation.

The MapReduce model was first pioneered [by Google](https://research.google/pubs/pub62/) in 2004 to resolve the practical problem of exponentially growing dataset for computing their search indexes. Then many have adopted and contributed toward the technology's development and evolution through the open-source community.

### Apache Spark™

Apache Spark is such an open-source framework that came around 2014 (10 years after the initial MapReduce research paper) that provides an elegant and unified abstraction to enable large-scale data processing that can efficiently utilize from multiple-cores of a single machine to "multiple-clouds".

For the sake of simplicity, we will demonstrate through its usage on a single machine. First, obtain the number of CPU cores:

In [10]:
import multiprocessing

CORES = multiprocessing.cpu_count()
CORES

12

Borrowing from [another previous part](./12-generate-data.ipynb), where we attempted to generate random and hashed device IDs:

In [11]:
# mostly from Part 12 - Generate data
from uuid import uuid4
from hashlib import sha1


def gen_device_ids(count: int = 20_000) -> list:
    device_ids = []
    for _ in range(count):
        device_ids.append(str(uuid4()))
    # hash
    return [sha1(x.encode()).hexdigest() for x in device_ids]

Let's generate a relatively large batch of device IDs, say 1 million times the number of `CORES`:

In [12]:
%time device_ids = gen_device_ids(count=1_000_000 * CORES)

CPU times: user 51.9 s, sys: 5.31 s, total: 57.2 s
Wall time: 57.4 s


Due to the iterative single process, the time it takes is quite long.

In [13]:
%time device_ids[:10]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


['febf7697c700bcf0e19c28b1694d9ade9daa94a4',
 '9b28c48c527505f964db5492da238af2dc49bf62',
 '86709d1d565d531f32b07f563826befd1a219690',
 '39c4a141f0ac58b7f6369af3fe23e82501b60f35',
 '180702a758db917758cd828918494c01a97e8757',
 '9af5eef60b00e5cfb2cf38f8fe3474ed09dd9a79',
 '0f131b59e1eeaa1afe51734b5b064754800e056c',
 '1f16df80860ed088b47e60da7c047e43cbd760e8',
 '2a7ae71c4e87825fab7333431cf8c91b6457bae4',
 '1afec761c3238d688b46be71a8e76b315b14dbf2']

But since the list is already in memory, it takes very little time to read them out.

Let's try the same device ID generation with Spark in a distributed manner. We will initiate a Spark session with a local "master" that takes advantage of the number of `CORES` we have obtained.

In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master(f'local[{CORES}]').getOrCreate()
spark

The Spark UI which runs on the `localhost` is handy for monitoring and more.

![Spark UI](https://user-images.githubusercontent.com/2837532/122995953-46a9c700-d378-11eb-84a7-50917d34b7be.png)

The first approach involves a core concept and building block of Spark which is known as Resilient Distributed Datasets (`RDD`s).

In [15]:
%%time

def mapper(count):
    return [(d,) for d in gen_device_ids(count)]

# a list of [1_000_000, ..., 1_000_000], where the length is the number of CORES
counts = [1_000_000] * CORES

rdd = spark.sparkContext.parallelize(counts).flatMap(mapper)

CPU times: user 1.78 ms, sys: 2.07 ms, total: 3.85 ms
Wall time: 193 ms


The action takes a context from the available Spark session, schedule to parallelize over a list of 1 million counts, where the list length is the number of `CORES`. Then we instruct the parallelization to map the list over a `mapper` function that takes the individual count and generate a list of device IDs (in the form of a single element tuple). The `.flatMap` method is a convenient layer to ensure the resulting dataset is flattened as a single-tier list of device IDs, instead of a list of (number of `CORES`) lists.

Notice the time it takes to schedule is negligible, as Spark takes a _lazy_ approach to preserve computing resources until the computation is really needed.

In [16]:
%time rdd.take(10)

CPU times: user 5.79 ms, sys: 2.83 ms, total: 8.62 ms
Wall time: 5.81 s


[('573bcd3a6fc34384927103af362376d5b0f4b8b4',),
 ('3a6ddce8cb2f313838fa45b5ad43589692b56031',),
 ('1e813510fe0b256af4efc38fd402a43ce83084bb',),
 ('12522da8b4b0fc4743bea819d87361219757d1b5',),
 ('ef92f7f3fe7649a7e6bc075f4087edab14dd56a0',),
 ('4a2f9b4ea5b39cc3d54ee0e14ffc53fa767a5250',),
 ('30493527bc351bca3a5846f65e780b6096b820e7',),
 ('30d2fc83f4ddc3daa040613a62ccac4ce6b7aced',),
 ('5ab40e264de56d041f7ad57c8ecfd3faabc4ced6',),
 ('de8af809e8bdb2658bee7c622709af5c0b0d318d',)]

Indeed, when we instruct to take the first 10 values from the RDD, Spark would actually start the computation (and more). If we go back to the Spark UI, it may paint a better picture on what Spark has done under the hood.

![RDD take](https://user-images.githubusercontent.com/2837532/122997519-16fbbe80-d37a-11eb-8704-5c520019e161.png)

Conceptually, the workflow is as the following:
1. Generate the device IDs leveraging multiple `CORES` in parallel.
2. Persist device IDs as RDD into distributed blocks on disk. They are also replicated automatically across available distributions to support further parallelized processes. Spark also records other necessary metadata such as order of values to ensure that computations that require strict ordering do not get affected.
3. Draw the first 10 values from the RDD files and populates into the Python process that runs this Notebook.

This means that while it is significantly faster to generate the device IDs compare to the single process iterative approach, it is also much slower to read it out as it involves a more complex trip to read from files while ensuring the order of values are intact.

Spark also has a built-in DataFrame implementation on top of the RDDs, providing neat abstractions such as SQL queries and more, similar to Pandas DataFrame.

In [17]:
%time df = rdd.toDF(['device_id'])

CPU times: user 358 ms, sys: 40.4 ms, total: 398 ms
Wall time: 6.45 s


This step reveals the trade-off more prominently, where the time it takes to operate on distributed dataset can be much more time consuming than its in-memory counterpart, where we can take the in-memory `list` of `device_ids` and convert it into a Pandas DataFrame:

In [18]:
import pandas as pd

In [19]:
%time pdf = pd.DataFrame(device_ids, columns=['device_id'])

pdf

CPU times: user 552 ms, sys: 196 ms, total: 749 ms
Wall time: 747 ms


,device_id
0,febf7697c700bcf0e19c28b1694d9ade9daa94a4
1,9b28c48c527505f964db5492da238af2dc49bf62
2,86709d1d565d531f32b07f563826befd1a219690
3,39c4a141f0ac58b7f6369af3fe23e82501b60f35
4,180702a758db917758cd828918494c01a97e8757
...,...
11999995,fd2a9f5bf95a26704c8dacd413d507c30e63d9a5
11999996,f5fadf65f12df3da07a871722a4f44af2cfb4be8
11999997,355434966437796f8b036f177b057ff609f2ec1e
11999998,9a15ae5f0b98ac9a0aaa043a190840bdbcf0d729


Bearing the same understanding, the Spark DataFrame would be slower to read, due to the round-trip it takes to the distributed disk blocks.

In [20]:
%time df.show()

+--------------------+
|           device_id|
+--------------------+
|2e9644239b7cc33e8...|
|251f4722f37e2a937...|
|ac9e04262f0c15b46...|
|a76b13cf7f71869a1...|
|6da357e917f3da496...|
|4cc9f4a9bceae8baa...|
|4774aaad84c9ce2ee...|
|9c6c39454d0d0769c...|
|22434c0bf67e5f9cc...|
|10e1aa81878ea6e60...|
|4df5812cd9d2e5d43...|
|05f94b9226b0c83a6...|
|3cd56c7020ccfa814...|
|b589f8a926c3115e1...|
|fb042723528fc5f4f...|
|50e1e59e40b3d3361...|
|7d705e2801fd5d065...|
|43234eb1c0f7ead35...|
|6a108f0b194f77b4a...|
|fbd4fc33e217e8128...|
+--------------------+
only showing top 20 rows

CPU times: user 1.15 ms, sys: 996 µs, total: 2.15 ms
Wall time: 5.56 s


In [21]:
%time pdf.count()

CPU times: user 396 ms, sys: 8.18 ms, total: 404 ms
Wall time: 403 ms


device_id    12000000
dtype: int64

In [22]:
%time df.count()

CPU times: user 2.28 ms, sys: 1.83 ms, total: 4.12 ms
Wall time: 16.7 s


12000000

Taking the full count can be more significant of a difference. Spark would attempt to perform underlying optimization to not scan the full range of dataset when invoking partial readings such as `rdd.take()` or `df.show()`, but suffers a full-range scan when it needs to count the accurate number of items, hence the much longer duration.

Spark DataFrames can also be created directly, or through the help of concatinating existing Pandas DataFrames.

Below is an alternative approach to perform the device ID generation task by leveraging the robust support of Spark's native support to interface with Pandas DataFrame.

In [23]:
df = spark.createDataFrame([(i,) for i in range(CORES)], ['cluster'])

def _gen(df):
    device_ids = gen_device_ids(count=1_000_000)
    pdf = pd.DataFrame(device_ids, columns=['device_id'])
    pdf['cluster'] = df['cluster']
    return pdf.reset_index()

def gen_device_ids_udf(df):
    output = []
    for _, row in df.iterrows():
        pdf = _gen(df)
        output.append(pdf)

    return pd.concat(output)


schema = 'index long, cluster long, device_id string'
%time df = df.groupby('cluster').applyInPandas(gen_device_ids_udf, schema=schema).drop('cluster', 'index')

CPU times: user 43.5 ms, sys: 5.25 ms, total: 48.7 ms
Wall time: 218 ms


The similar parallelization principle from the RDD approach applies here, with a _hack_ around the mechanism of `.groupby()` which enables the parallelization against the number fo `CORES`.

Also similarly, the scheduling of the task does not take much, as we have yet to request for actual access to the data.

In [24]:
%time df.show()

+--------------------+
|           device_id|
+--------------------+
|b3624be1446a9c651...|
|620a012fe70bbff6b...|
|02c22a78668015932...|
|50c3e622a52400eab...|
|246247ae209b744bb...|
|0d0e0bb66b184f47c...|
|70e0e67aac25517c6...|
|7a87d9a32ae988dd8...|
|9ac88faf90644db27...|
|260e1206125cbb6c8...|
|cd646c9c613f0902d...|
|af26d6ed80e434ecc...|
|a967561fc441f1951...|
|dd6a428cc42ed5a49...|
|032ffa42748cce517...|
|d8037c4744e21b142...|
|6f2052eb2db8868cb...|
|c0dae01dfadaf24ec...|
|4d5c20e70117653df...|
|4bc967f393258b90b...|
+--------------------+
only showing top 20 rows

CPU times: user 3.3 ms, sys: 2.82 ms, total: 6.12 ms
Wall time: 6.7 s


Since this approach involves a more pronounced mapping process abstracted by the `.groupby` method, underlying workflow can be a bit more interesting to observe:

![group by](https://user-images.githubusercontent.com/2837532/123001100-5b895900-d37e-11eb-8b2a-db3fa40caaba.png)

In [25]:
%time df.count()

CPU times: user 15.8 ms, sys: 9.95 ms, total: 25.8 ms
Wall time: 15.3 s


12000000

Read access and the time consumption behaviours are within our expectations.

Let's attempt to perform some actual analysis of the overall dataset, such as counting the number of device IDs by their first characters.

We'll start with the Pandas DataFrame, which is in-memory.

In [26]:
pdf['first'] = pdf.device_id.astype(str).str[0]
pdf

,device_id,first
0,febf7697c700bcf0e19c28b1694d9ade9daa94a4,f
1,9b28c48c527505f964db5492da238af2dc49bf62,9
2,86709d1d565d531f32b07f563826befd1a219690,8
3,39c4a141f0ac58b7f6369af3fe23e82501b60f35,3
4,180702a758db917758cd828918494c01a97e8757,1
...,...,...
11999995,fd2a9f5bf95a26704c8dacd413d507c30e63d9a5,f
11999996,f5fadf65f12df3da07a871722a4f44af2cfb4be8,f
11999997,355434966437796f8b036f177b057ff609f2ec1e,3
11999998,9a15ae5f0b98ac9a0aaa043a190840bdbcf0d729,9


In [27]:
import time

size_time = []
size = len(pdf)

while size > 1:
    sample = pdf.sample(size)
    start = time.time()
    sample.groupby('first').agg({'device_id': 'count'})
    end = time.time()
    size_time.append({
        'size': size,
        'time': end - start,
    })
    size = size // 100

pdf_size_time = pd.DataFrame(size_time)
pdf_size_time

,size,time
0,12000000,2.354549
1,120000,0.030982
2,1200,0.002792
3,12,0.001873


Even in-memory, the time it takes to aggregate the counts is not negligible, and it would obviously get worse with larger size.

Depending on the available computing resources, this may vary. But after certain threashold, it exhibits a linear growth of time needed to perform the operation.

Now let's try with Spark.

In [28]:
df = df.withColumn('first', df.device_id.substr(0, 1))
df.show()

+--------------------+-----+
|           device_id|first|
+--------------------+-----+
|8819657ee2ae716ae...|    8|
|775fff96ef0b57250...|    7|
|efc4765433e9c2b9f...|    e|
|e857d474fca33eeda...|    e|
|3af20fda97011257e...|    3|
|58f0196ec6c39ca0d...|    5|
|b71299272dd289954...|    b|
|df88e50b836d6ec62...|    d|
|564563cae645631ca...|    5|
|a90df3138c0c90656...|    a|
|3dfcafba601f0f840...|    3|
|0674924f56cf0a573...|    0|
|ad107f4631dd6b533...|    a|
|8622654408bf70128...|    8|
|ef032047843cae893...|    e|
|d6601aeb82b8cc91b...|    d|
|e37e34c064b16535d...|    e|
|24114557b4a7e9b10...|    2|
|30dc00cf907d5a3b7...|    3|
|097129d6e3e6ae525...|    0|
+--------------------+-----+
only showing top 20 rows



From above examples, we already know that Spark DataFrames, due to an entirely different and more complex mechanism compared to the more direct in-memory model that Pandas employs, the sampling process may be more time consuming. Therefore the runtime of the entire while loop logic may take much longer, but the aggregation portion is captured precisely like its Pandas counterpart.

In [29]:
%%time

size_time = []
count = df.count()
size = count

while size > 1:
    sample = df.sample(size / count)
    _count = sample.count()
    start = time.time()
    sample.groupby('first').agg({'device_id': 'count'}).collect()  # collect is used to emulate full data scan/process
    end = time.time()
    size_time.append({
        'size': _count,
        'time': end - start,
    })
    size = size // 100

df_size_time = pd.DataFrame(size_time)
df_size_time

CPU times: user 183 ms, sys: 111 ms, total: 294 ms
Wall time: 2min 29s


,size,time
0,12000000,17.799956
1,119741,16.251669
2,1210,17.157911
3,10,15.165370


While it is significantly slower to bootstrap the samples, the aggregation shows a glimpse of Spark's true strength (or the distributed filesystem and the MapReduce model behind it). 

The time it takes to compute the aggregation is nearly uniform regardless of the given sample size. This scalability characteristic becomes more prominent, and also more essential as a tool, to process data when the sample size goes beyond a single machine's capacity, much like the situation that Google encountered in the early 2000s.

In [30]:
# locus example placeholder

## Remarks

On a single machine, depending on the tasks, we can leverage techniques such as multiprocessing, multithreading, or coroutines to leverage underlying computing resources to expedite processing time.

When the data size is much more than a single machine can handle, tools such as Apache Spark and Modin (and its underlying parallization abstractions Dask and Ray, all mentioned in [the previous part](./13-data-processing.ipynb)) become more essential to get the job done in a timely and cost-effective manner.

For instance, at EQ Works, we leverage a cloud provider managed Spark service to run through terabytes to petabytes of data with variance of computational complexities in hours or minutes which would otherwise take days or even months on a single machine, even if the said machine _can_ hold such amount of data reliably.

![EMR](https://user-images.githubusercontent.com/2837532/123005848-c89fed00-d384-11eb-8967-2e78faf718f2.png)

## References

* [Apache Spark](https://spark.apache.org/) and its [PySpark interface](https://spark.apache.org/docs/latest/api/python/index.html)
* [Python Coroutines and Tasks](https://docs.python.org/3/library/asyncio-task.html)
* [Part 11 - Work with SQL](./11-work-with-sql.ipynb)
* [Part 12 - Generate Data](./12-generate-data.ipynb)
* [Part 13 - The Power of Parallel Processing feat. multithreading and multiprocessing](./13-data-processing.ipynb)